# LIME Explanation for image data 

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,DirectoryIterator
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K
import os
import tensorflow as tf
from tensorflow.keras.applications import inception_v3 as inc_net
# from skimage import io
from tensorflow.keras.preprocessing import image

path  =r'./data/images/animals'
# Path to train and test directory
dir_ = os.path.join(path)


# Generate training and test data with Image Generator
train_datagen = ImageDataGenerator(rescale=1/255, validation_split = 0.2)


train_generator = train_datagen.flow_from_directory(dir_,target_size=(200, 200),
                                                   batch_size= 2400,
                                                   class_mode='categorical',
                                                   shuffle=False,
                                                   subset = 'training')

test_generator = train_datagen.flow_from_directory(dir_,target_size = (200,200),
                                                  batch_size = 600,
                                                  class_mode = 'categorical',
                                                  shuffle=False,
                                                  subset = 'validation')


# Fetch the data and the labels
xtrain, ytrain = next(train_generator)
xtest, ytest  = next(test_generator)

# Fix the filepath
test_filepath = []
for filepath in test_generator.filepaths:
    filepath = filepath.replace('\\', '/')
    test_filepath.append(filepath)


/usr/local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


## Training the black box model as a CNN network with 3 Conv layers

In [ ]:
cnn_model = Sequential([
    # First convolution
        Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
        MaxPooling2D(2, 2),
    # Second convolution
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
    # Third convolution
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
    # Dense hidden layer
        Dense(512, activation='relu'),
        Dropout(0.2),
    # Output neuron. 
        Dense(3, activation='softmax')])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(name="precision"),tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.AUC(name='p-r-c', curve='PR')])
history = cnn_model.fit(train_generator, epochs=50, verbose=1)

cnn_model.save('./models/image/cnn_model_for_lime.h5')
cnn_model = tf.keras.models.load_model('./models/image/cnn_model_for_lime.h5')


Epoch 1/160
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-08-10 01:40:33.875 tensorflow-2-6-cpu--ml-r5-24xlarge-d37b532af3012f01b7914fb26ba7:14890 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-08-10 01:40:34.953 tensorflow-2-6-cpu--ml-r5-24xlarge-d37b532af3012f01b7914fb26ba7:14890 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
1/1 [==============================] - 31s 31s/step - loss: 1.0999 - precision: 0.0000e+00 - recall: 0.0000e+00 - p-r-c: 0.3431
Epoch 2/160
1/1 [==============================] - 23s 23s/step - loss: 1.5955 - precision: 0.5336 - recall: 0.5192 - p-r-c: 0.4803
Epoch 3/160
1/1 [===============

In [ ]:
from skimage.segmentation import mark_boundaries
from skimage import data, io
index_list = np.random.randint(len(xtest), size=9)
images= xtest[index_list, :, :, :]

fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(10,10))
i=0
for row in ax:
    for col in row:
        col.imshow(images[i])
        i+=1
plt.savefig('./result/dcp_animals.png', dpi=300)
plt.show()

In [ ]:
%%time
from lime import lime_image
from tensorflow.keras.preprocessing import image
class_names = {0:'cat', 1:'dog', 2:'panda'}

explainer = lime_image.LimeImageExplainer()
i = np.random.randint(len(xtest))
image= xtest[i:i+1, :, :, :]

model_output = cnn_model.predict(image)
predicted_label = np.argmax(model_output)
prob = np.max(model_output)
print(f'The CNN model predicted this image as: {class_names[predicted_label]} with probability of {prob:.2f}')

explanation = explainer.explain_instance(image[0].astype('double'), cnn_model.predict, top_labels=3, hide_color=0, num_samples=500)

# num_features is the number of superpixel (or number of image segmentations)
img, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
ax1.imshow(image[0])
ax2.imshow(mark_boundaries(img, mask))
ax1.axis('off')
ax2.axis('off')
plt.savefig(f'./result/lime_dcp_output{i}.png', dpi=300)


In [ ]:
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3
import json 
with open('./models/image/imagenet_class_index.json') as f:
    labels = json.load(f)

inception_model = InceptionV3(weights ='./models/image/inception_v3_weights_tf_dim_ordering_tf_kernels.h5')

#transforming the image to the appropriate format
def transform_img_to_inception_format(img):    
    img = skimage.transform.resize(img, (299,299))
    img = (img - 0.5)*2
    img = np.expand_dims(img, axis=0)
    preds = inception_model.predict(img)
    top=1
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(labels[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        print(result)
    return img, result[0]

image_transformed, image_result = transform_img_to_inception_format(image[0])


## Using the Pre-trained Microsoft ResNet model as a black box

In [ ]:
%%time
from PIL import Image
from transformers import AutoFeatureExtractor, TFResNetForImageClassification

print(f'The Inception_V3 model predicted this image as: {image_result[1]} with probability of {image_result[2]:.2f}')

explanation = explainer.explain_instance(image_transformed[0], inception_model.predict, top_labels=3, hide_color=0, num_samples=500)

# num_features is the number of superpixel (or number of image segmentations)
img, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
ax1.imshow(image[0])
ax2.imshow(mark_boundaries(img, mask))
ax1.axis('off')
ax2.axis('off')
plt.savefig(f'./result/lime_dcp_output{i}.png', dpi=300)


